In [ ]:
import cv2
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import Tracker

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = YOLO("yolov9c.pt")
tracker = Tracker()

target_classes = {2, 7} 
cap = cv2.VideoCapture("5473765-uhd_3840_2160_24fps.mp4")
count_vehicles = 0
counter_ids = set()


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 640))
    frame_tensor = torch.from_numpy(frame_resized.astype(np.float32)).to(device)
    frame_tensor = frame_tensor.permute(2, 0, 1).unsqueeze(0) / 255.0

    results = model.predict(frame_tensor)
    a = results[0].boxes.data.to("cpu").numpy()
    px = pd.DataFrame(a).astype("float")

    detected_objects = []
    for _, row in px.iterrows():
        x1, y1, x2, y2 = map(int, row[:4])
        class_id = int(row[5])

        if class_id in target_classes:
            detected_objects.append([x1, y1, x2 - x1, y2 - y1])

    bbox_id = tracker.update(detected_objects)

    y_line = 550
    offset = 7

    for bbox in bbox_id:
        x3, y3, w, h, obj_id = bbox
        cx = x3 + w // 2
        cy = y3 + h // 2

        cv2.circle(frame_resized, (cx, cy), 4, (0, 0, 255), -1)
        cv2.rectangle(frame_resized, (x3, y3), (x3 + w, y3 + h), (0, 255, 0), 2)

        if y_line - offset < cy < y_line + offset:
            if obj_id not in counter_ids:
                counter_ids.add(obj_id)
                count_vehicles += 1

    cv2.putText(
        frame_resized,
        f"Vehicles: {count_vehicles}",
        (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA,
    )

    cv2.line(frame_resized, (20, y_line), (640, y_line), (0, 0, 255), 3)
    

    frame_display = cv2.resize(frame_resized, (640, 360))
    cv2.imshow("Vehicle Counter", frame_display)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

Using device: cuda
